In [ ]:

from tkinter import filedialog

import cv2
import sys
import tkinter

def ProcessImage(self):
    OriginalImage = cv2.imread(filename, 1)
    cv2.imshow("Original Image", OriginalImage)
    b = OriginalImage[:, :, 0]
    g = OriginalImage[:, :, 1]
    r = OriginalImage[:, :, 2]
    cv2.imshow("Red Channel", r)
    cv2.imshow("Green Channel", g)
    cv2.imshow("Blue Channel", b)
    Disease = r - g
    global Alpha
    Alpha = b
    GetAlpha(OriginalImage)
    cv2.imshow("Alpha Channel", Alpha)
    ProcessingFactor = S.get()
    for i in range(0, OriginalImage.shape[0]):
        for j in range(0, OriginalImage.shape[1]):
            if int(g[i, j]) > ProcessingFactor:
                Disease[i, j] = 255
    cv2.imshow("Disease Image", Disease)
    DisplayDiseasePercentage(Disease)
    S.bind('<ButtonRelease-1>', ProcessImage)
    MainWindow.mainloop()


def GetAlpha(OriginalImage):
    global Alpha
    for i in range(0, OriginalImage.shape[0]):
        for j in range(0, OriginalImage.shape[1]):
            if OriginalImage[i, j, 0] > 200 and OriginalImage[i, j, 1] > 200 and OriginalImage[i, j, 2] > 200:
                Alpha[i, j] = 255
            else:
                Alpha[i, j] = 0


def GetFile():
    if len(sys.argv) > 1:
        return sys.argv[1]
    else:
        return filedialog.askopenfilename(title="Select Image")


def DisplayDiseasePercentage(Disease):
    Count = 0
    Res = 0
    for i in range(0, Disease.shape[0]):
        for j in range(0, Disease.shape[1]):
            if Alpha[i, j] == 0:
                Res += 1
            if Disease[i, j] < S.get():
                Count += 1
    Percent = (Count / Res) * 100
    DiseasePercent.set("Percentage Disease: " + str(round(Percent, 2)) + "%")


Alpha = None
MainWindow = tkinter.Tk()
MainWindow.title("Plant Disease Detector by Nahush Kulkarni")

S = tkinter.Scale(MainWindow, from_=0, to=255, length=500, orient=tkinter.HORIZONTAL,
                  background='white', fg='black', troughcolor='white', label="Processing Factor")
S.pack()
S.set(150)

DiseasePercent = tkinter.StringVar()
L = tkinter.Label(MainWindow, textvariable=DiseasePercent)
L.pack()

filename = GetFile()
if filename != "":
    ProcessImage(None)
else:
    print("No File!")
    exit(0)

In [1]:
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img,img_to_array
import numpy as np

model=load_model('Plant Disease Detection model Building and testing/models/potato_leaf_disease_classification_model.keras')
img_width,img_height=180,180
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

2023-09-29 13:49:52.239017: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-29 13:49:52.499208: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-29 13:49:52.501293: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-29 13:49:53.473919: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[ERROR:0@3.902] global persistence.cpp:512 open Can't open file: 'haarcascade_frontalface_default.xml' in read mode


In [5]:
cap=cv2.VideoCapture('/home/fansan/Videos/potato_disease/SaveTube.io-Potato Late Blight Preview Clip-(480p).mp4')
img_count_full=0
font=cv2.FONT_HERSHEY_SIMPLEX
org=(1,1)
class_label = ''
fontScale = 1
color = (255,0,0)
thickness = 2

while True:
    img_count_full+=1
    response,color_img = cap.read()
    if response == False:
        break
    scale=50
    width=int(color_img.shape[1]*scale/100)
    height=int(color_img.shape[0]*scale/100)
    dim = (width,height)
    color_img=cv2.resize(color_img,dim,interpolation=cv2.INTER_AREA)
    gray_img=cv2.cvtColor(color_img,cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(color_img,1.1,6)
    img_count=0
    
    for(x,y,w,h) in faces:
        org=(x-10, y-10)
        img_count+=1
        color_face=color_img[y:y+h,x:x+w]
        cv2.imwrite('input/%d%dface.jpg'%(img_count_full,img_count),color_face)
        img=load_img('input/%d%dface.jpg'%(img_count_full,img_count),target_size=(img_width,img_height))  
        img=img_to_array(img)
        img=np.expand_dims(img,axis=0)
        predictions=model.predict(img)
        
        if predictions==0:
            class_label = 'Late Blight'
            color = (255,0,0)
        elif predictions==1:
            class_label = 'Early Blight'
            color = (255,255,0)
        else:
            class_label = 'healthy'
            color = (0,255,0)
        cv2.rectangle(color_img,(x,y),(x+w,y+h),(255,0,0),3)
        cv2.putText(color_img,class_label,org,font,fontScale,color,thickness,cv2.LINE_AA)
        
    cv2.imshow('Plant Disease Detection',color_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
    

error: OpenCV(4.8.0) /io/opencv/modules/objdetect/src/cascadedetect.cpp:1689: error: (-215:Assertion failed) !empty() in function 'detectMultiScale'
